#### SymPrompt Focal Method Benchmark датасет

 **SymPrompt** - стратегия создания подсказок для языковых моделей (LLMs) с учетом кода, предназначенная для генерации тестов. SymPrompt опирается на подход, показывающий, что LLMs лучше решают сложные задачи, если процесс разбивается на несколько этапов с последовательными подсказками. Методология применена к генерации тестов путем разбиения процесса на многоступенчатую последовательность, где каждый этап включает соответствующую подсказку, основанную на путях выполнения метода, а также содержит информацию о типах и зависимостях.  

**SymPrompt** позволяет LLMs создавать более полные тесты без дополнительного обучения. Реализация базируется на **TreeSitter** и тестируется на сложных задачах из Python-проектов с открытым исходным кодом. Метод улучшает корректность тестов в 5 раз и увеличивает относительное покрытие тестов на 26% для CodeGen2. С использованием GPT-4 **SymPrompt** удваивает покрытие тестов по сравнению с базовыми подходами подсказок.

Посмотрим на данные из датасета

In [ ]:
import json
import pandas as pd

def parse_json_input(path_json = './data/data_sym_prompt_focal_method/focal_methods.jsonl'):
	'''Функция считываем файл формата JSON по пути и преобразует его в датафрейм pandas'''
	df = pd.read_json(path_json, lines=True)

	return df

code_dataset = parse_json_input()
code_dataset.head()


,prompt_id,project,module,class,method,focal_method_txt,focal_method_lines,in_stack,globals,type_context
0,0,apimd,apimd.loader,,walk_packages,"def walk_packages(name: str, path: str) -> Ite...","[43, 59]",False,"[__author__, __copyright__, __license__, __ema...",from typing import Optional\nfrom collections....
1,1,apimd,apimd.loader,,loader,"def loader(root: str, pwd: str, link: bool, le...","[78, 105]",False,"[__author__, __copyright__, __license__, __ema...",from typing import Optional\nfrom collections....
2,2,apimd,apimd.parser,,is_public_family,def is_public_family(name: str) -> bool:\n ...,"[61, 70]",False,"[__author__, __copyright__, __license__, __ema...","from typing import cast, TypeVar, Union, Optio..."
3,3,apimd,apimd.parser,,walk_body,def walk_body(body: Sequence[stmt]) -> Iterato...,"[73, 86]",False,"[__author__, __copyright__, __license__, __ema...","from typing import cast, TypeVar, Union, Optio..."
4,4,apimd,apimd.parser,,esc_underscore,"def esc_underscore(doc: str) -> str:\n """"""E...","[100, 105]",False,"[__author__, __copyright__, __license__, __ema...","from typing import cast, TypeVar, Union, Optio..."


Размерность датасета

In [10]:
print(f"Число сэмплов: {code_dataset.shape[0]}")
print(f"Число колонок в датасете: {code_dataset.shape[1]}")

Число сэмплов: 897
Число колонок в датасете: 10


На первый взгляд релевантными окажутся колонки:

In [ ]:
relevant_columns = ['method', 
                    'focal_method_txt',
                    'focal_method_lines',
                    'type_context']

Наиболее важные из них - focal_method_txt и type_context

Для первого сэмпла

* focal_method_txt:
	```python
	def walk_packages(name: str, path: str) -> Iterator[tuple[str, str]]:
    """Walk packages without import them."""
    path = abspath(path) + sep
    valid = (path + name, path + name + PEP561_SUFFIX)
    for root, _, fs in walk(path):
        for f in fs:
            if not f.endswith(('.py', '.pyi')):
                continue
            f_path = parent(join(root, f))
            if not f_path.startswith(valid):
                continue
            name = (f_path
                    .removeprefix(path)
                    .replace(PEP561_SUFFIX, "")
                    .replace(sep, '.')
                    .removesuffix('.__init__'))
            yield name, f_path
	```

* type_context: 
	```python
	from typing import Optional
	from collections.abc import Sequence, Iterator
	from sys import path as sys_path
	from os import mkdir, walk
	from os.path import isdir, isfile, abspath, join, sep, dirname
	from importlib.abc import Loader
	from importlib.machinery import EXTENSION_SUFFIXES
	from importlib.util import find_spec, spec_from_file_location, module_from_spec
	from .logger import logger
	from .parser import parent, Parser

	__author__ = "Yuan Chang"
	__copyright__ = "Copyright (C) 2020-2021"
	__license__ = "MIT"
	__email__ = "pyslvs@gmail.com"
	PEP561_SUFFIX = '-stubs'

	def walk_packages(name: str, path: str) -> Iterator[tuple[str, str]]:
		"""Walk packages without import them."""
		path = abspath(path) + sep
		valid = (path + name, path + name + PEP561_SUFFIX)
		for root, _, fs in walk(path):
			for f in fs:
				if not f.endswith(('.py', '.pyi')):
					continue
				f_path = parent(join(root, f))
				if not f_path.startswith(valid):
					continue
				name = (f_path
						.removeprefix(path)
						.replace(PEP561_SUFFIX, "")
						.replace(sep, '.')
						.removesuffix('.__init__'))
				yield name, f_path
	```



#### Резюме

* Небольшой датасет

* Релевантных для решаемой задачи вещей маловато... Лично я не увидел.